## Python Notebook for a paper upload to the SLED database 

In [79]:
import os
import json
import base64
import requests
from requests.auth import HTTPBasicAuth

from astropy import units as u
from astropy.coordinates import SkyCoord

### Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example in your shell profile file (e.g., in Unix/Linux/Mac include export SLED_PASSWORD=??? and export SLED_USERNAME=??? in your .bash_profile), or in an environment file.

In [73]:
urlquery = "http://127.0.0.1:8000/api/upload-lenses/"
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')

if (not username) or (not password):
    print('You have not set your SLED username and password environment variables. Please read the comments above.')

### Now we post the lens data to the database by sending the relevant information in a dictionary
Use the requests package to send a POST request to the relevant API endpoint. Some import notes:
1. The minimal required fields are name, ra, dec, mugshot (please see conventions here (TO BE LINKED TO SOME NOTES SOMEWHERE...))
2. The mugshot is required and the code below will convert the image path on your computer into a base 64 encoded image to be sent to the API endpoint

In [76]:
# Prepare the data
#How do import from a spreadsheet file like this...
# Prepare the data
lenses = [
    {
        'ra': 49.454167,  # Decimal field, between 0 and 360
        'dec': 42.003056,  # Decimal field, between -90 and 90
        'name': "Alternate Name A",  # CharField, max_length=100
        'alt_name': '',  # CharField, max_length=200
        'score': 3,  # DecimalField, between 0 and 3
        'image_sep': 1.84,  # DecimalField, between 0 and 100
        'info': 'This is an interesting lens system.',  # TextField
        'n_img': 4,  # IntegerField, between 2 and 20
        'imagename': 'J031749_mugshot.png',  # The name of the image file
        "flag": "CANDIDATE",
        "image_conf": ["QUAD", "FOLD"],
        "lens_type": ["GALAXY", "LRG"],
        "source_type": ["QUASAR"],
        "access_level": "PRI"
    }
]

# Function to convert image to base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


# Add base64 encoded images to the lenses data
for lens in lenses:
    lens['mugshot'] = image_to_base64(f"{lens['imagename']}")

# Convert the data to JSON
json_data = json.dumps(lenses)

# Set the headers
headers = { 'Content-Type': 'application/json'}

r = requests.post(urlquery, data=json_data, headers=headers,auth=HTTPBasicAuth(username, password))

dbquery = json.loads(r.text)

In [77]:
print(dbquery)

{'errors': {'non_form_errors': ['No changes detected.']}}


In [78]:
r

<Response [400]>

In [71]:
r.content

b'{"errors":{"non_form_errors":["No changes detected."]}}'